# Data Modeling with Apache Cassandra

## <font color=blue>Part I. ETL Pipeline for Pre-Processing the Files</font>

In [1]:
# Import Python packages 
import re
import os
import glob
import json
import csv

In [2]:
import numpy as np
import pandas as pd
from IPython.display import display

In [3]:
import cassandra
from cassandra.cluster import Cluster

In [4]:
from helper import *

#### Creating list of filepaths to process original event csv data files

In [5]:
# Print the current working directory
os.getcwd()

'/home/workspace'

In [6]:
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'
filepath

'/home/workspace/event_data'

In [7]:
# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

In [8]:
len(file_path_list) # 30

30

In [9]:
# Print sample path to the csv files.
file_path_list[:3]

['/home/workspace/event_data/2018-11-27-events.csv',
 '/home/workspace/event_data/2018-11-04-events.csv',
 '/home/workspace/event_data/2018-11-07-events.csv']

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [10]:
# Initiate an empty list of rows that will be generated from each file.
full_data_rows_list = [] 
    
for fp in file_path_list:
    # Read csv file.
    with open(fp, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        # Extract each data row one by one and append it.      
        for line in csvreader:
            full_data_rows_list.append(line) 

In [11]:
# Get total number of rows 
len(full_data_rows_list)

8056

In [12]:
# Check what the list of event data look like
full_data_rows_list[:2]

[['Barry Tuckwell/Academy of St Martin-in-the-Fields/Sir Neville Marriner',
  'Logged In',
  'Mohammad',
  'M',
  '0',
  'Rodriguez',
  '277.15873',
  'paid',
  'Sacramento--Roseville--Arden-Arcade, CA',
  'PUT',
  'NextSong',
  '1.54051E+12',
  '961',
  'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)',
  '200',
  '1.54328E+12',
  '88'],
 ['Jimi Hendrix',
  'Logged In',
  'Mohammad',
  'M',
  '1',
  'Rodriguez',
  '239.82975',
  'paid',
  'Sacramento--Roseville--Arden-Arcade, CA',
  'PUT',
  'NextSong',
  '1.54051E+12',
  '961',
  'Woodstock Inprovisation',
  '200',
  '1.54328E+12',
  '88']]

In [13]:
# Create a smaller event data csv file called event_datafile_full csv
##that will be used to insert data into the Apache Cassandra tables.
csv.register_dialect('myDialect', quoting= csv.QUOTE_ALL, skipinitialspace= True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect= 'myDialect')
    # Write the header row (column names) first.
    writer.writerow(['artist', 'firstName', 'gender', 'itemInSession', 'lastName',
                     'length', 'level', 'location', 'sessionId', 'song', 'userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6],
                         row[7], row[8], row[12], row[13], row[16]))

In [14]:
# # Check the number of rows in the generated csv file.
# with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
#     print(sum(1 for line in f))

In [15]:
# Preview the data.
df = pd.read_csv('event_datafile_new.csv')
df.shape

(6820, 11)

In [16]:
df.head()

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Horn Concerto No. 4 in E flat K495: II. Romanc...,88
1,Jimi Hendrix,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Woodstock Inprovisation,88
2,Building 429,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",961,Majesty (LP Version),88
3,The B-52's,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Love Shack,38
4,Die Mooskirchner,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",107,Frisch und g'sund,38


In [17]:
# Make sure the generated columns are in expected format.
# e.g., `length` is float ...
# N.B. when populating data, we use `csv` module,
# this will read every column as str, though.
df.dtypes

artist            object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
sessionId          int64
song              object
userId             int64
dtype: object

## <font color=blue>Part II. Complete the Apache Cassandra coding portion of your project.</font>

Now we are ready to work with the CSV file generated under the title <font color=red>event_datafile_new.csv</font>.

#### Creating a Cluster

In [18]:
# Make a connection to a Cassandra instance on local machine (127.0.0.1)
cluster = Cluster()

# Establish connection and begin executing queries; need a session.
session = cluster.connect()

#### Create Keyspace

In [19]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [20]:
# Set KEYSPACE to the keyspace specified above.
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

try:
    session.set_keyspace('udacity')
    session.row_factory = pandas_factory
except Exception as e:
    print(e)

Now we need to create tables to run the following queries. Remember, __with Apache Cassandra you model the database tables on the queries you want to run.__

#### 1. Return the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [21]:
# Decide table name for query 1.
tablename1 = 'sessions'

# Drop the table if exists.
drop_table(session, table_name= tablename1)

>>> Dropped <sessions> with success.


In [22]:
# As we know, distributed database design is all about queries.
# Especifically, the focus is on the WHERE clause.
# In the first query, first and foremost we want to filter by `session_id`
# this will become our partition key.
# Then, the filter is based on `itemInSession`, which will become our clustering key.
# Together, the partition key & the clustering key make the composite PRIMARY KEY.
query = (f"CREATE TABLE IF NOT EXISTS {tablename1} "
         "(session_id int, item_in_session int, artist text, song text, length float, "
         "PRIMARY KEY (session_id, item_in_session));")

create_table(session, table_name= tablename1, query= query)

>>> Table <sessions> created!


In [23]:
# Populate the table.
query = (f"INSERT INTO {tablename1} "
         "(session_id, item_in_session, artist, song, length) "
         "VALUES (%s, %s, %s, %s, %s);")
    
populate_table(session,
               file_path = 'event_datafile_new.csv',
               query= query,
               columns= (8, 3, 0, 9, 5))

>>> Populating the table.
Done.


In [24]:
# Fetch some query to make sure data is populated successfully.
query = (f"SELECT artist, song, length FROM {tablename1} "
         "WHERE session_id = 338 AND item_in_session = 4;")

try:
    rows = session.execute(query)
    df = rows._current_rows[['artist', 'song', 'length']]
    print('Query result: ' + '=' * 45)
    display(df)
except Exception as e:
    print(e)

Query result: =============================================


,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### 2. Return only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [25]:
# Decide the tablename for 2nd query.
tablename2 = 'user_session_item'

# Drop the table if exists.
drop_table(session, table_name= tablename2)

>>> Dropped <user_session_item> with success.


In [26]:
# Here the Primary Key has three fields:
# userid & sessionid are the partition keys,
# itemInSession is the clustering key. 
# Note: sessions belonging to the same user might be in different nodes.
# This will cause a performance issue if the database is very large.
# Therefore we should use both userId and sessionId as partition keys
# so that sessions from the same user are stored together.
query = (f"CREATE TABLE IF NOT EXISTS {tablename2} "
         "(user_id int, session_id int, item_in_session int, "
         "artist text, song text, first_name text, last_name text, "
         "PRIMARY KEY ((user_id, session_id), item_in_session));"
        )

create_table(session, table_name= tablename2, query= query)

>>> Table <user_session_item> created!


In [27]:
# Populate the table.
query = (f"INSERT INTO {tablename2} "
         "(user_id, session_id, item_in_session, artist, song, first_name, last_name) "
         "VALUES (%s, %s, %s, %s, %s, %s, %s);")
    
populate_table(session,
               file_path = 'event_datafile_new.csv',
               query= query,
               columns= (10, 8, 3, 0, 9, 1, 4))

>>> Populating the table.
Done.


In [28]:
query = (f"select artist, song, first_name, last_name from {tablename2} "
         "WHERE user_id = 10 and session_id = 182")

try:
    rows = session.execute(query)
    df = rows._current_rows[['artist', 'song', 'first_name', 'last_name']]
    print('Query result: ' + '=' * 45)
    display(df)
except Exception as e:
    print(e)

Query result: =============================================


,artist,song,first_name,last_name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### 3. Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [29]:
# Decide tablename for the 3rd query.
tablename3 = 'song_user'

# Drop the table if exists.
drop_table(session, table_name= tablename3)

>>> Dropped <song_user> with success.


In [30]:
# Here the Primary Key has two fields:
# song is the partition key, and userid are clustering keys. 
# Partitioning is done by song
# and within each partition, rows are ordered by the userid.
query = (f"CREATE TABLE IF NOT EXISTS {tablename3} "
         "(song text, user_id int, first_name text, last_name text, "
         "PRIMARY KEY (song, user_id));")

create_table(session, table_name= tablename3, query= query)

>>> Table <song_user> created!


In [31]:
# Populate the table.
query = (f"INSERT INTO {tablename3} "
         "(song, user_id, first_name, last_name) "
         "VALUES (%s, %s, %s, %s);")
    
populate_table(session,
               file_path = 'event_datafile_new.csv',
               query= query,
               columns= (9, 10, 1, 4))

>>> Populating the table.
Done.


In [32]:
# Fetch some query to make sure data is populated successfully.
query = (f"select first_name, last_name from {tablename3} "
         "WHERE song = 'All Hands Against His Own';")
try:
    rows = session.execute(query)
    df = rows._current_rows[['first_name', 'last_name']]
    print('Query result: ' + '=' * 45)
    display(df)
except Exception as e:
    print(e)

Query result: =============================================


,first_name,last_name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


In [33]:
query

"select first_name, last_name from song_user WHERE song = 'All Hands Against His Own';"

#### Drop the tables before closing out the sessions

In [34]:
drop_table(session, tablename1)
drop_table(session, tablename2)
drop_table(session, tablename3)

>>> Dropped <sessions> with success.
>>> Dropped <user_session_item> with success.
>>> Dropped <song_user> with success.


#### Close the session and cluster connection

In [35]:
session.shutdown()
cluster.shutdown()